Notebook elaborado para apresentar uma breve introdução ao processamento de sinais.
Criado para a disciplina Geofísica I da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>

<p>&nbsp;</p>
Última modificação: 25/02/2021

# Instruções iniciais

Neste notebook você irá aprender como a digitalização de um sinal analógico é feita corretamente a partir dos intervalos de amostragem. Para isto leia atentamente as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Primeiramente vamos preparar o notebook com os módulos e funções que precisaremos no decorrer da demonstração. A célula abaixo contém as funções utilizadas nos cálculos e as funções utilizadas para gerar as figuras. Execute-a uma vez antes de executar qualquer outra parte do código e **não altere nada no código apresentado na célula** para não criar problemas na execução do código.

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

<p>&nbsp;</p>

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada. 

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


class sinal_monofreq():
    
    """
    Sinal composto por uma única frequência e caracterizado por uma função cosseno na forma 
    y(t) = A * cos(2 * pi * F * t)
    """
    
    def __init__(self, a, f, tw):
        self.amplitude = a
        self.frequencia = f
        self.janela_tempo = tw
        self.dt = tw/1000
        
        # eixo de tempo estendido, apenas para evitar artefatos nas extremidades das interpolações
        self.tt_analogico = np.arange(-self.janela_tempo, (2*self.janela_tempo) + self.dt, self.dt)
        
        # Função de referência que caracteriza o sinal analógico na forma y(t) = A * cos(2 * pi * F * t).
        # Pode ser amostrada em qualquer instante de tempo que se queira, de forma que esta é a melhor
        # forma de representar um sinal contínuo para os objetivos desta demonstração.
        self.funcao_cos = interp1d(self.tt_analogico, self.amplitude * \
                                   np.cos(2*np.pi*self.frequencia*self.tt_analogico), kind='linear')      
        
        # Gera uma aproximação do sinal analógico calculando as amplitudes da função cosseno com inervalo
        # de amostragem curto o suficiente para que ela possa ser visualizada como um sinal analógico contínuo.
        self.analogico = self.funcao_cos(self.tt_analogico)
        
        # Inicializa com valores nulos pois não foi feita a amostragem ainda
        self.dt_amostrado = None
        self.tt_amostrado = None
        self.amostrado = None
        
        # O sinal recuperado usa o mesmo dt do sinal analógico original, mas aqui inicializa com valor nulo.
        self.recuperado = None
        self.tt_recuperado = None

   
    def amostragem(self, f_am):
        """
        Faz a amostragem do sinal calculando as amplitudes da função cosseno de acordo com o intervalo
        de amostragem definido.
        """
        
        t_min = self.tt_analogico[0]
        t_max = self.tt_analogico[-1]
        self.dt_amostrado = 1/f_am
        self.tt_amostrado = np.arange(t_min, t_max, self.dt_amostrado)

        # amostragem das amplitudes do sinal analógico
        self.amostrado = self.funcao_cos(self.tt_amostrado)      


    def recupera(self):
        """
        Interpola as amplitudes que foram amostradas da função cosseno para demonstrar como seria o
        comportamento real do sinal recuperado a partir destas amostras.
        """
        
        t_min = np.min(self.tt_amostrado)
        t_max = np.max(self.tt_amostrado)
        self.tt_recuperado = np.arange(t_min, t_max + self.dt, self.dt)
        
        funcao_recuperado = interp1d(self.tt_amostrado, self.amostrado, kind='cubic')
        self.recuperado = funcao_recuperado(self.tt_recuperado)

        
# funções para as figuras

def plota_amostragem(sinal, n_fig='X'):
    """
    Plota as amplitudes amostradas em relação ao sinal analógico original
    """
        
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(sinal.tt_analogico, sinal.analogico, '-r', label='Sinal original')
    ax.plot(sinal.tt_amostrado, sinal.amostrado, 'ob', label='Amplitudes amostradas')
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title("Figura " + n_fig + ". Sinal amostrado a uma frequência de " + \
                 str(1/sinal.dt_amostrado) + " amostras por segundo", fontsize=14)
    ax.set_xlim([0, sinal.janela_tempo])
    ax.set_ylim([np.min(sinal.analogico)*1.25, np.max(sinal.analogico)*1.25])
    ax.grid()
    ax.legend(loc='lower right', fontsize=12)

    
def plota_analogico(sinal, n_fig='X'):
    """
    Plota a função cosseno calculada a intervalos de tempo pequenos o suficiente para que se possa 
    fazer uma representação do sinal analógico original.
    """
    
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(sinal.tt_analogico, sinal.analogico)
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title("Figura " + n_fig + ". Representação do sinal analógico original", fontsize=14)
    ax.set_xlim([0, sinal.janela_tempo])
    ax.set_ylim([np.min(sinal.analogico)*1.25, np.max(sinal.analogico)*1.25])
    ax.grid()
    
    
def plota_representacao(sinal, n_fig='X'):
    """
    Plota a interpolação do sinal feita a partir das amostras obtidas do sinal analógico.
    """
    
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(sinal.tt_recuperado, sinal.recuperado, '--b', label='Sinal recuperado', linewidth=1)
    ax.plot(sinal.tt_amostrado, sinal.amostrado, 'ob', label='Amplitudes amostradas')
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title("Figura " + n_fig + ". Sinal recuperado da amostragem", fontsize=14)
    ax.set_xlim([0, sinal.janela_tempo])
    ax.set_ylim([np.min(sinal.analogico)*1.25, np.max(sinal.analogico)*1.25])
    ax.grid()
    ax.legend(loc='lower right', fontsize=12)
    
    
def plota_comparacao(sinal, n_fig='X'):
    """
    Plota a interpolação do sinal feita a partir das amostras obtidas do sinal 
    analógico e a sobrepõe à representação do sinal analógico original.
    """
    
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(sinal.tt_analogico, sinal.analogico, '-r', label='Sinal original')
    ax.plot(sinal.tt_recuperado, sinal.recuperado, '--b', label='Sinal recuperado')
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title("Figura " + n_fig + ". Comparação entre o sinal original e o sinal"\
                 " recuperado da amostragem", fontsize=14)
    ax.set_xlim([0, sinal.janela_tempo])
    ax.set_ylim([np.min(sinal.analogico)*1.25, np.max(sinal.analogico)*1.25])
    ax.grid()
    ax.legend(loc='lower right', fontsize=12)

### Passo 2 - Características do sinal analógico

Aqui iremos configurar as características que queremos no nosso sinal analógico. Nosso sinal analógico será caracterizado por uma função cosseno de amplitude *A*, dada em unidades de amplitude (ua) e uma única frequência *F*, dada em Hertz (Hz). Vamos começar com a amplitude de 1.0 ua e a frequência de 1.0 Hz. Este sinal será representado dentro de um intervalo finito de tempo, que iremos chamar de *janela de tempo* ou *janela temporal*, dada em segundos (s). Nossa janela temporal será inicialmente 20.0 s.

Execute a célula abaixo para estabelecer as configurações dos parâmetros do sinal analógico. Não altere nada por enquanto.

In [ ]:
amplitude = 1.0                  # amplitude do sinal em unidades de amplitude
frequencia = 1.0                 # frequência em Hertz
janela_tempo = 10.0              # janela temporal em segundos

Agora vamos gerar o sinal analógico propriamente dito. Entretanto, temos uma limitação aqui. O computador trabalha apenas com informações discretas, i.e., é preciso ter valores específicos de tempo e amplitude para plotar corretamente a figura, e isto por si só já caracteriza uma espécie de digitalização. Iremos contornar esta limitação criando nosso sinal analógico a partir de uma função cosseno definida matematicamente. Isto permite que possamos obter valores de amplitude em qualquer instante de tempo que queiramos, já que a função cosseno é uma função contínua. 

Execute a célula abaixo para obter a expressão matemática que representará nosso sinal analógico.

In [ ]:
meu_sinal = sinal_monofreq(amplitude, frequencia, janela_tempo)

Está achando tudo muito abstrato até aqui? Não se desespere, pois agora vamos gerar uma representação visual do sinal analógico. O que fazemos aqui é avaliar a nossa função cosseno em pontos que estão tão próximos entre si que podem ser interpolados sem prejuízo para a representação da sua continuidade.

Execute a célula abaixo para visualizar o sinal analógico.

In [ ]:
plota_analogico(meu_sinal, n_fig='1')

**Perguntas:** Este sinal corresponde ao que você esperava? As características de amplitude e frequência estão de acordo com os parâmetros configurados? Qual o período (em segundos) deste sinal?

### Passo 03 - Amostragem

Quando amostramos um sinal analógico estamos na verdade registrando a amplitude deste sinal em instantes de tempo específicos. Pense em um pêndulo oscilando. A distância entre a massa do pêndulo em relação ao seu ponto de repouso, i.e., a posição em que o pêndulo permanece quando não há oscilação, pode ser compreendida como a amplitude do nosso sinal. Quando o pêndulo está em oscilação nossos olhos observam um movimento contínuo e suave. Obter uma amostra do movimento do pêndulo equivale a bater uma foto sua. Será uma representação da amplitude do movimento em um instante de tempo específico. Assim, se batermos fotos em diferentes instantes de tempo e agruparmos todas em sequência teremos uma amostragem do movimento do pêndulo. Sistemas dedicados podem então reproduzir o movimento do pêndulo de forma fiel.

No entanto, a amostragem não pode ser feita de qualquer maneira. Se ela não for feita adequadamente, o sinal - ou o movimento do pêndulo - não será representado com fidelidade. Primeiramente, é necessário que as amostras sejam coletadas em instantes de tempo separados entre si por intervalos de tempo ($\Delta t$) regulares. Em seguida, deve-se obedecer ao critério estabelecido pela **Teoria da amostragem de Shannon**, que estabelece que a frequência de amostragem ($f_{am}$) deve ser maior ou igual a duas vezes a frequência ($f$) a ser amostrada, i.e.,:

$$f_{am} \geq 2*f$$

A frequência de amostragem correponde ao número de amostras obtidas por unidade de tempo. O intervalo de tempo entre cada amostra é então dado pelo intervalo de amostragem ($\Delta t_{am}$), que é o inverso da frequência de amostragem:

$$\Delta t_{am} = \frac{1}{f_{am}}$$


Assim, pelo Teorema da Amostragem, temos a definição do critério para garantir a amostragem correta do sinal:

$$\Delta t_{am} = \frac{1}{f_{am}} = \frac{1}{2 * f}$$

Vamos então definir esta frequência de amostragem para nosso sinal executando a célula abaixo:

In [ ]:
f_amostragem = 2.0 * frequencia          # frequência de amostragem

e, em seguida, executando a célula abaixo para visualizar as amplitudes que foram amostradas (pontos azuis) no nosso sinal analógico (linha vermelha):

In [ ]:
meu_sinal.amostragem(f_amostragem)
plota_amostragem(meu_sinal, n_fig='2')

**Pergunta:** Qual o intervalo de amostragem utilizado?

### Passo 04 - Recuperação do sinal

Quando o sinal é amostrado corretamente é possível recuperá-lo através de sistemas dedicados, que fogem ao escopo desta demonstração. Aqui exemplificaremos a recuperação do sinal através de uma simples interpolação.

Quando registramos as amostras de um sinal, as informações sobre a amplitude do sinal e sobre os intantes de tempo em que estas amplitudes foram medidas são tudo o que temos para trabalhar. Não há informação alguma sobre o comportamento do sinal, que pode ser uma função cosseno, uma função seno, uma sobreposição de ambas, ou qualquer outra coisa. Desta forma, vamos pegar as amostras que visualizamos na Figura 2 e interpolar um novo sinal a partir delas executando a célula abaixo:

In [ ]:
meu_sinal.recupera()
plota_representacao(meu_sinal, n_fig='3')

Assim como um software de análise de dados geofísicos não tem ideia sobre o comportamento do sinal verdadeiro, não há qulquer informação pré-determinada sobre as características do sinal original na interpolação realizada nesta demonstração. Perceba que, ainda assim, é possível recuperar um sinal com as características de uma função cosseno. Mas será que o sinal recuperado é fiel ao sinal original?

Execute a célula abaixo para visualizar uma compraração entre o sinal recuperado (em azul) e o sinal original (em vermelho).

In [ ]:
plota_comparacao(meu_sinal, n_fig='4')

**Pergunta:** Você considera que a recuperação foi bem feita? Qual a frequência do sinal original? Qual a frequência do sinal recuperado? 